**Potential Hypotheses:**
Likely just going to focus on either the 2020 or 2024 presidential election.

Electoral college and popular vote numbers by state: https://www.fec.gov/resources/cms-content/documents/2024presgeresults.pdf
- how does median age correlate with spending
- how does population density correlate spending / voter turnout
- how does median income correlate with chosen candidate


In [ ]:
from census import Census
from us import states